# MIS Kinect Video Capture Module

## Video Setting

In [1]:
import os
import cv2
import numpy as np
import pykinect_azure as pykinect
from pykinect_azure.k4a import calibration
from pykinect_azure.k4a import _k4a
import time

# file name setting
Date="1117"
TrackName = ["test","Arlequin", "Eusebins", "Floresla", "Pierrot", "Valse Nobles","Promenade"]
VideoType = ["","_above", "_above2","_short"]
file_name = TrackName[6]+VideoType[2]

video_folder="./sample/videos/"+Date+"/"
os.makedirs(video_folder, exist_ok=True)
json_folder="./sample/json/"+Date+"/"
os.makedirs(json_folder, exist_ok=True)

json_file = json_folder+file_name+".json"



# Initialize the library, if the library is not found, add the library path as an argument
pykinect.initialize_libraries(track_body=True)

# Modify camera configuration
device_config = pykinect.default_configuration
device_config.color_resolution = pykinect.K4A_COLOR_RESOLUTION_1080P
device_config.color_format = pykinect.K4A_IMAGE_FORMAT_COLOR_BGRA32
device_config.camera_fps = pykinect.K4A_FRAMES_PER_SECOND_30
device_config.depth_mode = pykinect.K4A_DEPTH_MODE_WFOV_2X2BINNED
device_config.synchronized_images_only =True
# print(device_config)


# output video setting
fps =30.0
fourcc = cv2.VideoWriter_fourcc(*'H264')  # 'H264' is the codec for MP4
out＿RGB = cv2.VideoWriter(video_folder+file_name+"_RGB"+".mp4", fourcc, fps, (1920, 1080))  # Adjust the resolution as needed
out_Ndepth = cv2.VideoWriter(video_folder+file_name+"_normalized_depth"+".mp4", fourcc, fps, (1920, 1080))  # Adjust the resolution as needed

# Start device
device = pykinect.start_device(config=device_config)

# Start body tracker
bodyTracker = pykinect.start_body_tracker()
# clamp 調遠方的點雲 rescaling 近處的
# Set the desired depth range for visualization
min_depth = 600   # Adjust this value based on your specific scenario
max_depth = 1200 # Adjust this value based on your specific scenario

# 問題不是FOV 而是distortion，depth不能直接拿來用
# 先去照能照到手的問題
# window_size = (1280, 720)
frame_idx = 0
t = time.time()
try:
    while True:
        # Get capture
        capture = device.update()
        # Get body tracker frame
        # body_frame = bodyTracker.update()
        # Get the color image
        ret_rgb, img_RGB = capture.get_color_image()
        #不會distortion 的 depth photo
        # ret_depth, img_depth = capture.get_transformed_colored_depth_image()
        ret_depth, img_depth = capture.get_transformed_depth_image()
        # ret_color, body_image_color = body_frame.get_segmentation_image()
        
        if not ret_rgb or not ret_depth:
            continue
        
        normalized_depth = np.clip(img_depth, min_depth, max_depth)
        normalized_depth = (normalized_depth - min_depth) / (max_depth - min_depth) * 255
        normalized_depth = normalized_depth.astype(np.uint8)
        # _img_depth= _img_depth.astype(np.uint8)
        # ret_ir, img_ir = capture.get_ir_image()

        
        # Display the color image
        cv2.imshow('MIS Kinect View RGB', img_RGB)
        # cv2.resizeWindow('MIS Kinect View RGB', *window_size)
        # cv2.imshow('MIS Kinect View Depth', img_depth)
        cv2.imshow('MIS Kinect View Normalized Depth', normalized_depth)
    
        # cv2.imshow('MIS Kinect View IR', img_ir)
        # video output
        out＿RGB.write(img_RGB)
        out_Ndepth.write(normalized_depth)
        # wait=cv2.waitKey(1000)
        # Press q key to stop
        key = cv2.waitKey(1)
        frame_idx+=1
        # print(frame_idx)
        if key == ord('q') or key == 27:  # Check for 'q' key or ESC key
            break

# except KeyboardInterrupt:
#     # Release the resources
#     print("FPS: ", frame_idx/(time.time()-t))
#     cv2.destroyAllWindows()
#     device.close()
#     out＿RGB.release()
#     out_Ndepth.release()
finally:
    # Release the resources
    print("FPS: ", frame_idx/(time.time()-t))
    cv2.destroyAllWindows()
    device.close()
    out＿RGB.release()
    out_Ndepth.release()

print("Video output finished!!")


FPS:  22.637730071021576
Video output finished!!


In [8]:
# import os
# import cv2
# import numpy as np
# import pykinect_azure as pykinect
# from pykinect_azure.k4a import calibration
# from pykinect_azure.k4a import _k4a
# import time

# # file name setting
# Date="1116"
# TrackName = ["test","Arlequin", "Eusebins", "Floresla", "Pierrot", "Valse Nobles"]
# VideoType = ["","_above", "_side","_short"]
# file_name = TrackName[0]+VideoType[0]

# video_folder="./sample/videos/"+Date+"/"
# os.makedirs(video_folder, exist_ok=True)
# json_folder="./sample/json/"+Date+"/"
# os.makedirs(json_folder, exist_ok=True)

# json_file = json_folder+file_name+".json"



# # Initialize the library, if the library is not found, add the library path as an argument
# pykinect.initialize_libraries(track_body=True)

# # Modify camera configuration
# device_config = pykinect.default_configuration
# device_config.color_resolution = pykinect.K4A_COLOR_RESOLUTION_1080P
# device_config.color_format = pykinect.K4A_IMAGE_FORMAT_COLOR_BGRA32
# device_config.camera_fps = pykinect.K4A_FRAMES_PER_SECOND_30
# device_config.depth_mode = pykinect.K4A_DEPTH_MODE_WFOV_2X2BINNED
# device_config.synchronized_images_only =True
# print(device_config)


# # output video setting
# fps =30.0
# fourcc = cv2.VideoWriter_fourcc(*'H264')  # 'H264' is the codec for MP4
# out＿RGB = cv2.VideoWriter(video_folder+file_name+"_RGB"+".mp4"
#                           , fourcc, fps, (1920, 1080))  # Adjust the resolution as needed
# out_Ndepth = cv2.VideoWriter(video_folder+file_name+"_normalized_depth"+".mp4"
#                             , fourcc, fps, (1920, 1080))  # Adjust the resolution as needed

# # Start device
# device = pykinect.start_device(config=device_config)

# # Start body tracker
# bodyTracker = pykinect.start_body_tracker()
# # clamp 調遠方的點雲 rescaling 近處的
# # Set the desired depth range for visualization
# min_depth = 900   # Adjust this value based on your specific scenario
# max_depth = 1200 # Adjust this value based on your specific scenario

# # 問題不是FOV 而是distortion，depth不能直接拿來用
# # 先去照能照到手的問題
# # window_size = (1280, 720)
# frame_idx = 0
# t = time.time()
# try: 
#     while True:
#         # Get capture
#         capture = device.update()
#         # Get body tracker frame
#         # body_frame = bodyTracker.update()
#         # Get the color image
#         ret_rgb, img_RGB = capture.get_color_image()
#         #不會distortion 的 depth photo
#         # ret_depth, img_depth = capture.get_transformed_colored_depth_image()
#         ret_depth, img_depth = capture.get_transformed_depth_image()
#         # ret_color, body_image_color = body_frame.get_segmentation_image()
        
#         # if not ret_rgb or not ret_depth:
#         #     continue
        
#         normalized_depth = np.clip(img_depth, min_depth, max_depth)
#         normalized_depth = (normalized_depth - min_depth) / (max_depth - min_depth) * 255
#         normalized_depth = normalized_depth.astype(np.uint8)
#         # _img_depth= _img_depth.astype(np.uint8)
#         # ret_ir, img_ir = capture.get_ir_image()

        
#         # Display the color image
#         cv2.imshow('MIS Kinect View RGB', img_RGB)
#         # cv2.resizeWindow('MIS Kinect View RGB', *window_size)
#         # cv2.imshow('MIS Kinect View Depth', img_depth)
#         cv2.imshow('MIS Kinect View Normalized Depth', normalized_depth)
        
#         # cv2.imshow('MIS Kinect View IR', img_ir)
#         # video output
#         # out＿RGB.write(img_RGB)
#         # out_Ndepth.write(normalized_depth)
#         # wait=cv2.waitKey(1000)
#         # Press q key to stop
#         # key = cv2.waitKey(1)
#         frame_idx+=1
#         print(frame_idx)

#         # time
#         # if key == ord('q') or key == 27:  # Check for 'q' key or ESC key
#         #     break
# except KeyboardInterrupt:
#     print("Video output finished!")
#     print("FPS: ", frame_idx/(time.time()-t))
#     cv2.destroyAllWindows()
#     device.close()
#     out＿RGB.release()
#     out_Ndepth.release()

    


In [7]:
# cv2.destroyAllWindows()
# device.close()
# out＿RGB.release()
# out_Ndepth.release()